# **Wageningen B-Series Propellers** 

#### **Parameters:**
- **Aᴇ/A₀**: Blade area ratio
- **C(Q) s,t,u,v**: Coefficient in the \(K_Q\) polynomial expression
- **C(T) s,t,u,v**: Coefficient in the \(K_T\) polynomial expression
- **D**: Propeller diameter
- **J**: Advance coefficient
- **K_Q**: Torque coefficient
- **K_T**: Thrust coefficient
- **n**: Propeller revolutions per second
- **P/D**: Pitch-diameter ratio
- **Q**: Propeller torque
- **Re**: Reynolds number
- **T**: Propeller thrust
- **t/c**: Thickness to cord ratio for propeller blades
- **V_A**: Speed of advance
- **Z**: Number of blades
- **η**: Open-water propeller efficiency
- **ρ**: Fluid density

#### **Parameters Required for KT KQ calculation:**
1. **J**, Advanced ratio (float)
  Formula: J = Va/nD
  VA Formula: VA = V⋅(1-wT)⋅0.5144

3. **P/D**, Blade area ratio (float)
  Formula: P/D


5. **Aᴇ/A₀**, Blade area ratio (float)
  Formula: - Aᴇ/A₀


7. **z**, Number of Blades (int)


**Calculate KT**: KT = ∑C s,t,u,v⋅(J^s)⋅((P/D)^t)⋅((Aᴇ/A₀)^u)⋅((z)^v)

**Calculate KQ**: KQ = ∑C s,t,u,v⋅(J^s)⋅((P/D)^t)⋅((Aᴇ/A₀)^u)⋅((z)^v)




#### **User inputs**:
1. V, speed in knots (int)
2. wT, wake fraction coefficient (float)
3. P, pitch in meters (float)
4. D, diameter in meters (float)
5. BAR or Aᴇ/A₀ , blade area ratio (float)
6. z, number of blades (int)


# **Libraries**

In [1]:
# Set up Matplotlib backend
import matplotlib
matplotlib.use('qtagg')  # Or 'qt5agg' for Qt-based backends
%matplotlib qt

# Import libraries
# Standard libraries
import time
from math import pi, log10

# Third-party libraries
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor
import mplcursors
import numpy as np
from tqdm import tqdm

# GUI libraries
import tkinter as tk
from tkinter import ttk, simpledialog, messagebox


# **Calculations**

In [2]:
'''***************************************************************************************************
                                        Propeller Calculator Class 
------------------------------------------------------------------------------------------------------
    Description: 
        This is based on the KT, KQ, and Efficiency curves for Wageningen B-series Propellers.
        The purpose is to compute key performance metrics (thrust, torque, and advance ratio)
        for marine propellers using polynomial equations and coefficients from Wageningen 
        B-series data.
        
    Parameters:
        - Aᴇ/A₀ Blade area ratio
        - C(Q) s,t,u,v: Coefficient in the K_Q polynomial expression
        - C(T) s,t,u,v: Coefficient in the K_T polynomial expression
        - D: Propeller diameter
        - J: Advance coefficient
        - K_Q: Torque coefficient
        - K_T: Thrust coefficient
        - n: Propeller revolutions per second
        - P/D: Pitch-diameter ratio
        - Q: Propeller torque
        - Re: Reynolds number
        - T: Propeller thrust
        - t/c: Thickness to cord ratio for propeller blades
        - V_A: Speed of advance
        - Z: Number of blades
        - η: Open-water propeller efficiency
        - ρ: Fluid density
------------------------------------------------------------------------------------------------------
***************************************************************************************************'''
class PropellerCalculator:
    def __init__(self):
        # Coefficients for K_T from the table
        self.KT_coefficients = [
            [0.00880496, 0, 0, 0, 0],
            [-0.204554, 1, 0, 0, 0],
            [0.166351, 0, 1, 0, 0],
            [0.158114, 0, 2, 0, 0],
            [-0.147581, 2, 0, 1, 0],
            [-0.481497, 1, 1, 1, 0],
            [0.415437, 0, 2, 1, 0],
            [0.0144043, 0, 0, 0, 1],
            [-0.0530054, 2, 0, 0, 1],
            [0.0143481, 0, 1, 0, 1],
            [0.0606826, 1, 1, 0, 1],
            [-0.0125894, 0, 0, 1, 1],
            [0.0109689, 1, 0, 1, 1],
            [-0.133698, 0, 3, 0, 0],
            [0.00638407, 0, 6, 0, 0],
            [-0.00132718, 2, 6, 0, 0],
            [0.168496, 3, 0, 1, 0],
            [-0.0507214, 0, 0, 2, 0],
            [0.0854559, 2, 0, 2, 0],
            [-0.0504475, 3, 0, 2, 0],
            [0.010465, 1, 6, 2, 0],
            [-0.00648272, 2, 6, 2, 0],
            [-0.00841728, 0, 3, 0, 1],
            [0.0168424, 1, 3, 0, 1],
            [-0.00102296, 3, 3, 0, 1],
            [-0.0317791, 0, 3, 1, 1],
            [0.018604, 1, 0, 2, 1],
            [-0.00410798, 0, 2, 2, 1],
            [-0.000606848, 0, 0, 0, 2],
            [-0.0049819, 1, 0, 0, 2],
            [0.0025983, 2, 0, 0, 2],
            [-0.000560528, 3, 0, 0, 2],
            [-0.00163652, 1, 2, 0, 2],
            [-0.000328787, 1, 6, 0, 2],
            [0.000116502, 2, 6, 0, 2],
            [0.000690904, 0, 0, 1, 2],
            [0.00421749, 0, 3, 1, 2],
            [0.0000565229, 3, 6, 1, 2],
            [-0.00146564, 0, 3, 2, 2],
        ]
        self.KQ_coefficients = [
            [0.00379368, 0, 0, 0, 0],
            [0.00886523, 2, 0, 0, 0],
            [-0.032241, 1, 1, 0, 0],
            [0.00344778, 0, 2, 0, 0],
            [-0.0408811, 0, 1, 1, 0],
            [-0.108009, 1, 1, 1, 0],
            [-0.0885381, 2, 1, 1, 0],
            [0.188561, 0, 2, 1, 0],
            [-0.00370871, 1, 0, 0, 1],
            [0.00513696, 0, 1, 0, 1],
            [0.0209449, 1, 1, 0, 1],
            [0.00474319, 2, 1, 0, 1],
            [-0.00723408, 2, 0, 1, 1],
            [0.00438388, 1, 1, 1, 1],
            [-0.0269403, 0, 2, 1, 1],
            [0.0558082, 3, 0, 1, 0],
            [0.0161886, 0, 3, 1, 0],
            [0.00318086, 1, 3, 1, 0],
            [0.015896, 0, 0, 2, 0],
            [0.0471729, 1, 0, 2, 0],
            [0.0196283, 3, 0, 2, 0],
            [-0.0502782, 0, 1, 2, 0],
            [-0.030055, 3, 1, 2, 0],
            [0.0417122, 2, 2, 2, 0],
            [-0.0397722, 0, 3, 2, 0],
            [-0.00350024, 0, 6, 2, 0],
            [-0.0106854, 3, 0, 0, 1],
            [0.00110903, 3, 3, 0, 1],
            [-0.000313912, 0, 6, 0, 1],
            [0.0035985, 3, 0, 1, 1],
            [-0.00142121, 0, 6, 1, 1],
            [-0.00383637, 1, 0, 2, 1],
            [0.0126803, 0, 2, 2, 1],
            [-0.00318278, 2, 3, 2, 1],
            [0.00334268, 0, 6, 2, 1],
            [-0.00183491, 1, 1, 0, 2],
            [0.000112451, 3, 2, 0, 2],
            [-0.0000297228, 3, 6, 0, 2],
            [0.000269551, 1, 0, 1, 2],
            [0.00083265, 2, 0, 1, 2],
            [0.00155334, 0, 2, 1, 2],
            [0.000302683, 0, 6, 1, 2],
            [-0.0001843, 0, 0, 2, 2],
            [-0.000425399, 0, 3, 2, 2],
            [0.0000869243, 3, 3, 2, 2],
            [-0.0004659, 0, 6, 2, 2],
            [0.0000554194, 1, 6, 2, 2],
        ]
    def calculate_advance_ratio(self, V, wT, D, n):
        """
        Calculate advance ratio (J).
        J = Va / (n * D)
        """
        Va = V * (1 - wT) * 0.514444  # Convert knots to m/s
        return Va / (n * D)

    def calculate_KT(self, J, P_D, AE_AO, z):
        """
        Calculate thrust coefficient (K_T)
        """
        KT = 0
        for coeff, s, t, u, v in self.KT_coefficients:
            KT += coeff * (J ** s) * (P_D ** t) * (AE_AO ** u) * (z ** v)
        return KT

    def calculate_KQ(self, J, P_D, AE_AO, z):
        """
        Calculate torque coefficient (K_Q)
        """
        KQ = 0
        for coeff, s, t, u, v in self.KQ_coefficients:
            KQ += coeff * (J ** s) * (P_D ** t) * (AE_AO ** u) * (z ** v)
        return KQ

    def calculate(self, V, wT, P, D, AE_AO, z, n):
        """
        Main calculation
        """
        P_D = P / D
        J = self.calculate_advance_ratio(V, wT, D, n)
        KT = self.calculate_KT(J, P_D, AE_AO, z)
        KQ = self.calculate_KQ(J, P_D, AE_AO, z)
        return KT, KQ, J



## **User Query GUI**

In [3]:
'''***************************************************************************************************
                                    Launch User Query GUI
------------------------------------------------------------------------------------------------------
    Description: The InputGUI class is a graphical user interface (GUI) designed to gather user input
                 parameters for the Propeller Calculator. It provides a user-friendly interface to 
                 input key variables required for propeller performance calculations. This class 
                 ensures inputs are validated and formatted correctly before being used in computations.
***************************************************************************************************'''
class InputGUI:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Propeller Input Parameters")

        # Create labels and entry fields
        self.entries = {}
        self.labels = {
            "wT": "Wake fraction (wT):",
            "D": "Propeller diameter (D in meters):",
            "Ae_Ao": "Blade area ratio (Ae/Ao):",
            "Z": "Number of blades (Z):",
            "rho": "Fluid density (ρ in kg/m³):",
            "V": "Vessel speed (V in knots):",
            "PD_array": "P/D ratios (comma-separated):",
        }
        self.default_values = {
            "wT": "0",
            "D": "0.85",
            "Ae_Ao": "0.5",
            "Z": "4",
            "rho": "1023.6",
            "V": "10",
            "PD_array": "0.5, 0.6, 0.7, 0.8",
        }

        row = 0
        for key, label in self.labels.items():
            tk.Label(self.root, text=label, anchor="w").grid(row=row, column=0, padx=10, pady=5, sticky="w")
            self.entries[key] = tk.Entry(self.root, width=30)
            self.entries[key].insert(0, self.default_values[key])
            self.entries[key].grid(row=row, column=1, padx=10, pady=5)
            row += 1

        # Add Submit button
        self.submit_button = tk.Button(self.root, text="Submit", command=self.submit)
        self.submit_button.grid(row=row, column=0, columnspan=2, pady=10)

        self.root.mainloop()

    def submit(self):
        try:
            # Get and validate user inputs
            self.wT = float(self.entries["wT"].get())
            self.D = float(self.entries["D"].get())
            self.Ae_Ao = float(self.entries["Ae_Ao"].get())
            self.Z = int(self.entries["Z"].get())
            self.rho = float(self.entries["rho"].get())
            self.V = float(self.entries["V"].get())
            self.PD_array = [float(x.strip()) for x in self.entries["PD_array"].get().split(",")]

            # Close the GUI window
            self.root.destroy()
        except ValueError as e:
            messagebox.showerror("Input Error", f"Invalid input: {e}")



## **Reference Box GUI**

In [4]:
'''***************************************************************************************************
                                        RPM Reference Box GUI
------------------------------------------------------------------------------------------------------
    Description: The RPMCalculatorGUI class provides a graphical user interface (GUI) for calculating 
                 and displaying key performance metrics for a propeller system based on user-specified 
                 inputs. It integrates with the PropellerCalculator class to compute parameters such 
                 as thrust, torque, efficiencies, and power requirements, for marine propulsion
                 analysis.
***************************************************************************************************'''
class RPMCalculatorGUI:
    def __init__(self, master, propeller_calculator, hover_data_ax1, hover_data_ax2, hover_data_ax3, D, rho, V, wT, Ae_Ao, Z, PD_array):
        self.master = master
        self.master.title("RPM Reference")

        self.propeller_calculator = propeller_calculator
        self.hover_data_ax1 = hover_data_ax1
        self.hover_data_ax2 = hover_data_ax2
        self.hover_data_ax3 = hover_data_ax3

        self.D = D
        self.rho = rho
        self.V = V
        self.wT = wT
        self.Ae_Ao = Ae_Ao
        self.Z = Z
        self.PD_array = PD_array

        # Input labels and fields
        tk.Label(master, text="Enter RPM:", anchor='w').grid(row=0, column=0, padx=10, pady=10, sticky='w')
        self.entry_rpm = tk.Entry(master, width=15)
        self.entry_rpm.grid(row=0, column=1, padx=10, pady=10, sticky='w')
        
        tk.Label(master, text="Number of Propellers:", anchor='w').grid(row=1, column=0, padx=10, pady=10, sticky='w')
        self.entry_num_propellers = tk.Entry(master, width=15)
        self.entry_num_propellers.grid(row=1, column=1, padx=10, pady=10, sticky='w')

        tk.Label(master, text="Towing Resistance, Rt (kN):", anchor='w').grid(row=2, column=0, padx=10, pady=10, sticky='w')
        self.entry_Rt = tk.Entry(master, width=15)
        self.entry_Rt.grid(row=2, column=1, padx=10, pady=10, sticky='w')
        
        tk.Label(master, text="Sea Margin (%):", anchor='w').grid(row=3, column=0, padx=10, pady=10, sticky='w')
        self.entry_sea_margin = tk.Entry(master, width=15)
        self.entry_sea_margin.grid(row=3, column=1, padx=10, pady=10, sticky='w')
        
        tk.Label(master, text="Thrust Deduction (%):", anchor='w').grid(row=4, column=0, padx=10, pady=10, sticky='w')
        self.entry_thrust_deduction = tk.Entry(master, width=15)
        self.entry_thrust_deduction.grid(row=4, column=1, padx=10, pady=10, sticky='w')
        
        tk.Label(master, text="Relative Rotative Efficiency, ηr (%):", anchor='w').grid(row=5, column=0, padx=10, pady=10, sticky='w')
        self.entry_n_r = tk.Entry(master, width=15)
        self.entry_n_r.grid(row=5, column=1, padx=10, pady=10, sticky='w')
        
        tk.Label(master, text="Shaft Efficiency, ηs (%):", anchor='w').grid(row=6, column=0, padx=10, pady=10, sticky='w')
        self.entry_n_s = tk.Entry(master, width=15)
        self.entry_n_s.grid(row=6, column=1, padx=10, pady=10, sticky='w')
        
        tk.Label(master, text="Engine Efficiency, ηe (%):", anchor='w').grid(row=7, column=0, padx=10, pady=10, sticky='w')
        self.entry_n_e = tk.Entry(master, width=15)
        self.entry_n_e.grid(row=7, column=1, padx=10, pady=10, sticky='w')

        # Output area
        self.output_text = tk.Text(master, height=20, width=70)
        self.output_text.grid(row=8, column=0, columnspan=2, padx=10, pady=10)

        # Calculate button
        self.calculate_button = tk.Button(master, text="Calculate", command=self.calculate)
        self.calculate_button.grid(row=9, column=0, columnspan=2, pady=10)

    def calculate(self):
        try:
            rpm = float(self.entry_rpm.get())
            sea_margin = float(self.entry_sea_margin.get()) / 100 if self.entry_sea_margin.get() else 0
            thrust_deduction = float(self.entry_thrust_deduction.get()) / 100 if self.entry_thrust_deduction.get() else 0
            num_propellers = int(self.entry_num_propellers.get()) if self.entry_num_propellers.get() else 1

            Rt = float(self.entry_Rt.get()) if self.entry_Rt.get() else 0  # kN
            n_r = float(self.entry_n_r.get())/100 if self.entry_n_r.get() else 1.0
            n_s = float(self.entry_n_s.get())/100 if self.entry_n_s.get() else 1.0
            n_e = float(self.entry_n_e.get())/100 if self.entry_n_e.get() else 1.0

            rps = rpm / 60  # Convert to RPS

            # Calculate total drag
            total_drag = Rt + (Rt * sea_margin) + (Rt * thrust_deduction)  # in kN

            # Drag per propeller
            drag_per_propeller = total_drag / num_propellers if num_propellers > 0 else 0

            # Rt from user, let's use Rt to compute Pe:
            # Convert V (knots) to m/s
            V_mps = self.V * 0.5144
            # Convert Rt (kN) to N
            Rt_N = Rt * 1000
            # Effective power from Rt: Pe = Rt * V (in W) -> convert to kW
            Pe = (Rt_N * V_mps) / 1000  # kW
            
            # Part 1
            result_text = "------------------------------------------------------------------\n\t\t\tDefault Parameters\n------------------------------------------------------------------\n"
            result_text += f"  Wake fraction (wT): {self.wT}\n"
            result_text += f"  Propeller diameter (D): {self.D} meters\n"
            result_text += f"  Blade area ratio (Ae/Ao): {self.Ae_Ao}\n"
            result_text += f"  Number of blades (Z): {self.Z}\n"
            result_text += f"  Fluid density (ρ): {self.rho} kg/m³\n"
            result_text += f"  Vessel speed (V): {self.V} knots\n"
            result_text += f"  P/D ratios: {', '.join(map(str, self.PD_array))}\n\n"
            
            # Part 2
            result_text += f"------------------------------------------------------------------\n\t\t\tUser Inputs\n------------------------------------------------------------------\n"
            result_text += f"  RPM: {rpm:.2f}\n  RPS: {rps:.2f}\n\n"
            result_text += f"  User Inputs:\n"
            result_text += f"  Total Drag from user: {total_drag:.2f} kN\n"
            result_text += f"  Rt (Towing Resistance): {Rt:.2f} kN\n"
            result_text += f"  Sea Margin: {sea_margin*100:.1f}%\n"
            result_text += f"  Thrust Deduction: {thrust_deduction*100:.1f}%\n"
            result_text += f"  Relative Rotative Eff (ηr): {n_r*100:.1f}%\n"
            result_text += f"  Shaft Eff (ηs): {n_s*100:.1f}%\n"
            result_text += f"  Overall Prop Eff (ηe): {n_e*100:.1f}%\n"
            result_text += f"  Number of Propellers: {num_propellers}\n\n"
            result_text += f"  Effective Power (Pe) from Rt: {Pe:.2f} kW\n\n"
            
            # Compute hull efficiency n_h = (1 - t)/(1 - wT)
            # t = thrust_deduction
            # wT = self.wT (already known)
            if (1 - self.wT) != 0:
                n_h = (1 - thrust_deduction)/(1 - self.wT)
            else:
                n_h = 1.0  # Fallback if wT=1 which is unusual

            # Now for each P/D ratio compute n_o (open-water efficiency) and then n_p and n_tot
            for PD in self.PD_array:
                P = self.D * PD
                KT, KQ, J = self.propeller_calculator.calculate(self.V, self.wT, P, self.D, self.Ae_Ao, self.Z, rps)
                # Open-water efficiency (n_o)
                # Efficiency formula from code: efficiency = (KT/(KQ*10))*(J/(2*pi)) (This was your open-water approx)
                n_o = (KT/(KQ))*(J/(2*np.pi)) if KQ > 0 else 0

                Thrust = KT * (self.rho * (rps**2) * (self.D**4)) # N
                Torque = KQ * (self.rho * (rps**2) * (self.D**5)) # Nm

                # Propeller Efficiency n_p = n_h * n_o * n_r * n_s
                n_p = n_h * n_o * n_r * n_s

                # Total Efficiency n_tot = n_p * n_e
                n_tot = n_p * n_e

                # Compute Required Engine Power
                P_engine = Pe / n_tot if n_tot > 0 else 0
                
                result_text += (
                    f"\n------------------------------------------------------------------\n\t\t\t    @P/D {PD:.2f}:\n------------------------------------------------------------------\n"
                    f"  Plot Results [KT, KQ, J]: [{KT:.3f}, {KQ:.3f}, {J:.3f}]\n"
                    f"  Thrust: {Thrust:.2f} N\n"
                    f"  Torque: {Torque:.2f} Nm\n"
                    f"  η values (h, o, r, s, e): [{n_h:.2f}, {n_o:.2f}, {n_r:.2f}, {n_s:.2f}, {n_e:.2f}, {n_p:.2f}]\n"
                    f"  ηp (Prop): {n_p*100:.2f}%\n"
                    f"  ηtot (Total): {n_tot*100:.2f}%\n"
                    f"  P_engine for {num_propellers:.0f} Propellers: {P_engine:.2f} kW\n\n"
                )

            # Display results
            self.output_text.delete("1.0", tk.END)
            self.output_text.insert(tk.END, result_text)

        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter valid inputs.")


# ***CODE***

In [ ]:
def main():
    plt.ioff()
    
    '''***************************************************************************************************
                                            Get User Inputs
    ***************************************************************************************************'''
    gui = InputGUI()
    wT = gui.wT
    D = gui.D
    Ae_Ao = gui.Ae_Ao
    Z = gui.Z
    rho = gui.rho
    V = gui.V
    PD_array = gui.PD_array 
    n_values = np.linspace(1, 1000, 50000)

    # Initialize PropellerCalculator()
    propeller_calculator = PropellerCalculator()
    n_values = np.linspace(1, 1000, 50000)

    
    '''***************************************************************************************************
                                            Loading Window Setup
    ***************************************************************************************************'''
    # Setup the Tkinter loading window
    loading_root = tk.Tk()
    loading_root.title("Loading...")

    total_steps = len(PD_array) * len(n_values)

    progress_label = tk.Label(loading_root, text="Calculating and plotting, please wait...")
    progress_label.pack(pady=10)

    progress_var = tk.DoubleVar()
    progress_bar = ttk.Progressbar(loading_root, orient="horizontal", length=300, mode="determinate", variable=progress_var, maximum=100)
    progress_bar.pack(pady=10)

    loading_root.update()

    current_step = 0
    update_interval = 10000  # Update the UI every 1000 steps

    
    '''***************************************************************************************************
                                            Plot Configuration
    ***************************************************************************************************'''
    fig = plt.figure(figsize=(16, 12))
    ax1 = fig.add_subplot(2, 2, 1)
    ax1.set_title("KT (Thrust Coefficient)")
    ax1.set_ylabel("KT (Thrust Coefficient)", color="blue")
    ax1.tick_params(axis="y", labelcolor="blue")
    ax1.set_ylim(0, 1)
    ax1.set_yticks(np.arange(0, 1.2, 0.2))
    ax1.set_xlabel("J (Advance Coefficient)")
    ax1.set_xlim(0, 1.4)
    ax1.set_xticks(np.arange(0, 1.6, 0.2))
    ax1.grid()

    ax2 = fig.add_subplot(2, 2, 2)
    ax2.set_title("KQ (Torque Coefficient)")
    ax2.set_ylabel("KQ (Torque Coefficient)", color="green")
    ax2.tick_params(axis="y", labelcolor="green")
    ax2.set_ylim(0, 0.2)
    ax2.set_yticks(np.arange(0, 0.24, 0.04))
    ax2.set_xlabel("J (Advance Coefficient)")
    ax2.set_xlim(0, 1.4)
    ax2.set_xticks(np.arange(0, 1.6, 0.2))
    ax2.grid()

    ax3 = fig.add_subplot(2, 2, 4)
    ax3.set_title("Efficiency")
    ax3.set_ylabel("Efficiency", color="orange")
    ax3.tick_params(axis="y", labelcolor="orange")
    ax3.set_xlim(0, 1.4)
    ax3.set_xticks(np.arange(0, 1.6, 0.2))
    ax3.grid()

    hover_data_ax1 = []
    hover_data_ax2 = []
    hover_data_ax3 = []

    colors = cm.viridis(np.linspace(0, 1, len(PD_array)))
    most_efficient_points = []


    '''***************************************************************************************************
                                            Data Calculation Loop
    ***************************************************************************************************'''
    for i, PD in enumerate(PD_array):
        P = D * PD
        KT_values, KQ_values, J_values, efficiency_values = [], [], [], []

        for idx, n in enumerate(n_values):
            KT, KQ, J = propeller_calculator.calculate(V, wT, P, D, Ae_Ao, Z, n)
            KT_values.append(KT)
            KQ_values.append(KQ)
            J_values.append(J)
            efficiency = (KT / KQ) * (J / (2 * np.pi)) if KQ > 0 else 0
            efficiency_values.append(efficiency)

            current_step += 1

            # Update UI only every 'update_interval' steps
            if current_step % update_interval == 0 or current_step == total_steps:
                progress = (current_step / total_steps) * 100
                progress_var.set(progress)
                progress_bar["value"] = progress
                loading_root.update()  # Update the GUI

        max_efficiency = max(efficiency_values)
        max_index = efficiency_values.index(max_efficiency)
        J_max = J_values[max_index]
        n_max = n_values[max_index]
        RPM_max = n_max * 60
        KT_max = KT_values[max_index]
        KQ_max = KQ_values[max_index]
        Thrust_max = KT_max * (rho * (n_max**2) * (D**4))
        Torque_max = KQ_max * (rho * (n_max**2) * (D**5))

        most_efficient_points.append({
            "P/D": PD,
            "J_max": J_max,
            "RPM_max": RPM_max,
            "Efficiency_max": max_efficiency,
            "Thrust_max": Thrust_max,
            "Torque_max": Torque_max
        })

        color = colors[i]

        line_kt, = ax1.plot(J_values, KT_values, label=f"P/D={PD:.1f}", linestyle=":", color=color, linewidth=1)
        hover_data_ax1.append({
            "line": line_kt,
            "J_values": J_values,
            "KT_values": KT_values,
            "P/D": PD,
            "n_values": n_values
        })

        line_kq, = ax2.plot(J_values, KQ_values, label=f"P/D={PD:.1f}", linestyle="--", color=color, linewidth=1)
        hover_data_ax2.append({
            "line": line_kq,
            "J_values": J_values,
            "KQ_values": KQ_values,
            "P/D": PD,
            "n_values": n_values
        })

        line_eff, = ax3.plot(J_values, efficiency_values, label=f"P/D={PD:.1f}", linestyle="-", color=color, linewidth=1)
        hover_data_ax3.append({
            "line": line_eff,
            "J_values": J_values,
            "Efficiency_values": efficiency_values,
            "P/D": PD,
            "n_values": n_values
        })


    '''***************************************************************************************************
                                            Post Calculation Data Presentation
    ***************************************************************************************************'''
    loading_root.destroy()

    ax1.legend(loc="upper left")
    ax2.legend(loc="upper left")
    ax3.legend(loc="upper left")

    header = f"{'P/D':<8} {'Efficiency':<14} {'J':<12} {'RPM':<8} {'Thrust (N)':<14} {'Torque (Nm)':<14}\n"
    separator = "-" * 58 + "\n"
    summary_text = "Most Efficient Points:\n\n" + header + separator
    
    for point in most_efficient_points:
        summary_text += (
            f"{point['P/D']:<8.2f} "
            f"{point['Efficiency_max']:<14.3f} "
            f"{point['J_max']:<10.3f} "
            f"{point['RPM_max']:<8.0f} "
            f"{point['Thrust_max']:<14.2f} "
            f"{point['Torque_max']:<14.2f}\n"
        )
    
    fig.text(
        0.1, 0.4,
        summary_text,
        fontsize=10,
        color="black",
        verticalalignment="top",
        horizontalalignment="left",
        bbox=dict(facecolor="white", alpha=0.7, edgecolor="black"),
    )

    efficiency_values_all = [value for data in hover_data_ax3 for value in data["Efficiency_values"] if value > 0]
    if efficiency_values_all:
        ax3.set_ylim(0, max(efficiency_values_all) * 1.1)
    else:
        ax3.set_ylim(0, 1)

    cursor_ax1 = mplcursors.cursor([d["line"] for d in hover_data_ax1], hover=True)
    cursor_ax2 = mplcursors.cursor([d["line"] for d in hover_data_ax2], hover=True)
    cursor_ax3 = mplcursors.cursor([d["line"] for d in hover_data_ax3], hover=True)

    
    '''***************************************************************************************************
                                                Hover Cursor
    ***************************************************************************************************'''
    @cursor_ax1.connect("add")
    def on_add_ax1(sel):
        label = sel.artist.get_label()
        n_value = None

        for data in hover_data_ax1:
            if sel.artist == data["line"]:
                J_values = data["J_values"]
                KT_values = data["KT_values"]
                tolerance = 1e-2
                for i, J in enumerate(J_values):
                    if abs(J - sel.target[0]) < tolerance and abs(KT_values[i] - sel.target[1]) < tolerance:
                        n_value = data["n_values"][i]
                        break
                break

        Thrust = (sel.target[1]) * (rho * (n_value**2) * (D**4)) if n_value else None
        RPM = n_value * 60 if n_value else None
        sel.annotation.set_text(
            f"$\\mathbf{{{label}}}$\n\nKT: {sel.target[1]:.3f}\nJ: {sel.target[0]:.3f}\nRPM: {RPM:.0f}\nThrust(N): {Thrust:.2f}" if n_value else
            f"$\\mathbf{{{label}}}$\n\nKT: {sel.target[1]:.3f}\nJ: {sel.target[0]:.3f}\nRPM: NA\nThrust(N): NA"
        )

    @cursor_ax2.connect("add")
    def on_add_ax2(sel):
        label = sel.artist.get_label()
        n_value = None
        for data in hover_data_ax2:
            if sel.artist == data["line"]:
                J_values = data["J_values"]
                KQ_values = data["KQ_values"]
                tolerance = 1e-2
                for i, J in enumerate(J_values):
                    if abs(J - sel.target[0]) < tolerance and abs(KQ_values[i] - sel.target[1]) < tolerance:
                        n_value = data["n_values"][i]
                        break
                break

        Torque = (sel.target[1]) * (rho * (n_value**2) * (D**5)) if n_value else None
        RPM = n_value * 60 if n_value else None
        sel.annotation.set_text(
            f"$\\mathbf{{{label}}}$\n\nKQ: {sel.target[1]:.3f}\nJ: {sel.target[0]:.3f}\nRPM: {RPM:.0f}\nTorque(Nm): {Torque:.2f}" if n_value else
            f"$\\mathbf{{{label}}}$\n\nKQ: {sel.target[1]:.3f}\nJ: {sel.target[0]:.3f}\nRPM: NA\nTorque(Nm): NA"
        )

    @cursor_ax3.connect("add")
    def on_add_ax3(sel):
        label = sel.artist.get_label()
        n_value = None
        for data in hover_data_ax3:
            if sel.artist == data["line"]:
                J_values = data["J_values"]
                Efficiency_values = data["Efficiency_values"]
                tolerance = 1e-2
                for i, J in enumerate(J_values):
                    if abs(J - sel.target[0]) < tolerance and abs(Efficiency_values[i] - sel.target[1]) < tolerance:
                        n_value = data["n_values"][i]
                        break
                break

        RPM = n_value * 60 if n_value else None
        sel.annotation.set_text(
            f"$\\mathbf{{{label}}}$\n\nEfficiency: {sel.target[1]:.3f}\nJ: {sel.target[0]:.3f}\nRPM: {RPM:.0f}" if n_value else
            f"$\\mathbf{{{label}}}$\n\nEfficiency: {sel.target[1]:.3f}\nJ: {sel.target[0]:.3f}\nRPM: NA"
        )
    
    '''***************************************************************************************************
                                                    END
    ***************************************************************************************************'''
    plt.show()

    root = tk.Tk()
    app = RPMCalculatorGUI(root, propeller_calculator, hover_data_ax1, hover_data_ax2, hover_data_ax3, D, rho, V, wT, Ae_Ao, Z, PD_array)
    root.mainloop()

# Run the application
main()
